In [ ]:
import * as tslab from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf-8");
tslab.display.html(`<style>${css}</style>`);

# How to Check that a Formula is a Tautology

In this notebook we develop a function <tt>tautology</tt> that takes a formula $f$ from propositional logic and checks whether $f$ is a tautology.  As we represent tautologies as nested tuples, we first have to import the parser for propositional logic.

In [ ]:
import { LogicParser, Formula } from './PropositionalLogicParser'

In [ ]:
type Variable = string;
type Formula  = Variable | ['⊤' | '⊥'] | ['¬', Formula] | ['↔' | '→' | '∧' | '∨', Formula, Formula];

In [ ]:
function parse(s: string): Formula {
    const parser = new LogicParser(s);
    return parser.parse();
}

`Element` is a generic type variable that describes the type of the elements of a set.

As we represent propositional valuations as sets of variables, we need a function to compute all subsets of a given set.  The module <tt>power</tt> provides a function called <tt>allSubsets</tt> such that for a given set $M$ the function call $\texttt{allSubsets}(M)$ computes an array containing all subsets of $M$, that is we have:
$$ \texttt{allSubsets}(M) = \bigl[A \mid A \in 2^M\bigr] $$

In [ ]:
function allSubsets<Element>(M: Set<Element>): Array<Set<Element>> {
    const arr = Array.from(M);
    const result: Array<Set<Element>> = [];
    const n = arr.length;
    for (let i = 0; i < (1 << n); i++) {
        const subset = new Set<Element>();
        for (let j = 0; j < n; j++) {
            if ((i & (1 << j)) !== 0) {
                subset.add(arr[j]);
            }
        }
        result.push(subset);
    }
    return result;
}

In [ ]:
console.log(allSubsets(new Set(['p', 'q']))); 

In [ ]:
console.log(allSubsets(new Set(Array.from({length: 23}, (_, i) => i + 1))));

In [ ]:
console.log(allSubsets(new Set(['p', 'q', 'r'])));

To be able to compute all propositional valuations for a given formula $f$ we first need to determine the set of all variables that occur in $f$.  The function $\texttt{collectVars}(f)$ takes a formula $f$ from propositional logic and computes all propositional variables occurring in $f$.  This function is defined recursively.

In [ ]:
function collectVars(f: Formula): Set<string> {
    if (typeof f === 'string') {
        return new Set([f]);
    }
    if (f[0] === '⊤' || f[0] === '⊥') {
        return new Set();
    }
    if (f[0] === '¬') {
        return collectVars(f[1]);
    }
    if (f.length === 3) {
        const leftVars = collectVars(f[1]);
        const rightVars = collectVars(f[2]);
        return new Set([...leftVars, ...rightVars]);
    }
    return new Set();
}

We have discussed the function <tt>evaluate</tt> previously.  The call 
$\texttt{evaluate}(f, I)$ takes a propsitional formula $f$ and a propositional valuation $I$, where $I$ is represented as a set of propositional variables.  It evaluates $f$ given $I$.

In [ ]:
function evaluate(F: Formula, I: Set<string>): boolean {
    if (typeof F === 'string')
        { return I.has(F);}
    if (F[0] === '⊤')   return true;
    if (F[0] === '⊥')   return false;
    if (F[0] === '¬') { return !evaluate(F[1], I);}
    if (F[0] === '∧') { return  evaluate(F[1], I) &&   evaluate(F[2], I);}
    if (F[0] === '∨') { return  evaluate(F[1], I) ||   evaluate(F[2], I);}
    if (F[0] === '→') { return !evaluate(F[1], I) ||   evaluate(F[2], I);}
    if (F[0] === '↔') { return  evaluate(F[1], I) ===  evaluate(F[2], I);}
    return false;
} 

Now we are ready to define the function $\texttt{tautology}(f)$ that takes a propositional formula $f$ and checks whether $f$ is a tautology.  If $f$ is a tautology, the function returns <tt>true</tt>, otherwise a set of variables $I$ is returned such that $f$ evaluates to <tt>false</tt> if all variables in $I$ are <tt>true</tt>, while all variables not in $I$ are <tt>false</tt>.

In [ ]:
function tautology(f: Formula): true | Set<string> {
    // Check, whether the formula f is a tautology
    const P = collectVars(f);
    for (const I of allSubsets(P)) {
        if (!evaluate(f, I)) {
            return I;
        }
    }
    return true;
}

The function $\texttt{test}(s)$ takes a string $s$ that can be parsed as a propositional formula and checks whether this formula is a tautology.

In [ ]:
function test(s: string): void {
    const f = parse(s);
    const counterExample = tautology(f);
    if (counterExample === true) {
        console.log('The formula', s, 'is a tautology.');
    } else {
        const P = collectVars(f);
        console.log('The formula', s, 'is not a tautology.');
        console.log('Counter example:');
        for (const x of P) {
            if (counterExample.has(x)) {
                console.log(x, '↦ True');
            } else {
                console.log(x, '↦ False');
            }
        }
    }
}

Let us run a few tests.

The first example is DeMorgan's rule.

In [ ]:
test('¬(p ∨ q) ↔ ¬p ∧ ¬q');

In [ ]:
test('(p → q) → (¬p → q) → q');

In [ ]:
test('(p → q) → (¬p → ¬q)');

In [ ]:
test('(p → q) → (¬q → ¬p)');

In [ ]:
test('¬p ↔ (p → ⊥)');